In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os

In [3]:
!pip install keras-unet

In [4]:
import keras
from keras_unet.models import custom_unet

2023-03-08 12:40:22.218102: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/home/kunal.jain001/miniconda3/envs/O/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [5]:
DATAPATH = '/raid/mpsych/RISTERLAB/fly_segmentation_experiments/classified_images/npy_files'
images_file = os.path.join(DATAPATH, 'fly-mar-06-original-images.npy')
labels_file = os.path.join(DATAPATH, 'fly-mar-06-segmented.npy')

In [6]:
images = np.load(images_file)
labels = np.load(labels_file)

In [7]:
images.shape

(56, 512, 512)

In [8]:
# full dataset does not have the last channel
images = images.reshape(images.shape[0],images.shape[1],images.shape[2],1)
labels = labels.reshape(labels.shape[0],labels.shape[1],labels.shape[2],1)

In [9]:
images.shape

(56, 512, 512, 1)

In [10]:
p = np.random.permutation(len(images))

In [11]:
images = images[p]
labels = labels[p]

In [12]:
labels = labels.astype(np.float64)

In [13]:
images = images.astype(np.float64)

for i in range(images.shape[0]):
    images[i] = (images[i] - images[i].min()) / (images[i].max() - images[i].min()) # normalize individually

In [14]:
images.shape

(56, 512, 512, 1)

In [15]:
X_train = images[0:40]
y_train = labels[0:40]
X_val = images[40:50]
y_val = labels[40:50]
X_test = images[50:]
y_test = labels[50:]

In [16]:
model = custom_unet(
    input_shape=(512, 512, 1),
    use_batch_norm=False,
    num_classes=1,
    filters=32,
    dropout=0.5,
    output_activation='sigmoid')

2023-03-08 12:40:25.181603: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-03-08 12:40:25.210500: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-08 12:40:25.210525: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: chimera13
2023-03-08 12:40:25.210530: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: chimera13
2023-03-08 12:40:25.210588: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 450.172.1
2023-03-08 12:40:25.210603: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 450.172.1
2023-03-08 12:40:25.210607: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 450.172.1
2023-03-08 12:40:25.211034: I tensorflow/core/platform/cpu_fe

In [17]:
import keras.optimizers
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance

In [18]:
opt = keras.optimizer_v1.Adam(lr=0.01)

In [19]:
model.compile(optimizer = 'Adam',    
              loss='binary_crossentropy', 
              metrics=[iou, iou_thresholded])

In [ ]:
history = model.fit(X_train, 
                    y_train, 
                    32, 
                    epochs=100,
                    validation_data=(X_val, y_val), 
                    verbose=1)

2023-03-08 12:40:26.006030: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-03-08 12:40:26.007306: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2245645000 Hz


Epoch 1/100


In [ ]:
model.save(os.path.join(DATAPATH, '03-08.hdf5'))

In [ ]:
from keras_unet.utils import plot_segm_history

plot_segm_history(history)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from keras_unet.utils import plot_imgs

plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=10)

In [ ]:
X_test_new = X_test
y_test_new = y_test

In [ ]:
loaded_model = keras.models.load_model(os.path.join(DATAPATH, '03-08.hdf5'))

In [ ]:
y_pred_new = loaded_model.predict(X_test_new)

plot_imgs(org_imgs=X_test_new, mask_imgs=y_test_new, pred_imgs=y_pred_new, nm_img_to_plot=10)